In [ ]:
from openai import OpenAI
from config import config
client = OpenAI(api_key=config.openai_api_key)

In [ ]:
def fetch_product_description(item_name: str, item_category: str) -> str:
    """
    Fetches product description based on item name and category.
    """

            # Make API call with web search enabled
    response = client.responses.create(
            store = False,
            model = "gpt-4.1-mini",
            tools = [{"type": "web_search_preview"}],
            input = [{"role": "system", "content": f"Generate a product description for {item_name} in the category {item_category}."},
                      {"role": "user", "content": f"item_name: {item_name}, item_category: {item_category}"}],
            max_tokens = 1000,
            stream = False  # Enable web search capability
        )
    return response.choices[0].message.content